In [1]:
from creating_arrays import CHART_TIME_ARRAY, MONTH_ARRAY, SYMBOL_ARRAY
from constants import BINANCE_URL, download_dir, output_dir, dtypes
from pathlib import Path
import os
import re
import shutil
from selenium import webdriver
import talib
import datetime
from selenium.webdriver.chrome.options import Options
import glob
import zipfile
import pandas as pd
from pandas.api.types import is_numeric_dtype
import inspect
import talib
import time
import numpy as np

['1000BTTCUSDT', '1000LUNCBUSD', '1000LUNCUSDT', '1000SHIBBUSD', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'AKROUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'AMBBUSD', 'ANCBUSD', 'ANCUSDT', 'ANKRUSDT', 'ANTUSDT', 'APEBUSD', 'APEUSDT', 'API3USDT', 'APTBUSD', 'APTUSDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT', 'AUCTIONBUSD', 'AUDIOUSDT', 'AVAXBUSD', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLUEBIRDUSDT', 'BLZUSDT', 'BNBBUSD', 'BNBUSDT', 'BNXUSDT', 'BTCBUSD', 'BTCBUSD_210129', 'BTCBUSD_210226', 'BTCDOMUSDT', 'BTCSTUSDT', 'BTCUSDT', 'BTCUSDT_210326', 'BTCUSDT_210625', 'BTCUSDT_210924', 'BTCUSDT_211231', 'BTCUSDT_220325', 'BTCUSDT_220624', 'BTCUSDT_220930', 'BTCUSDT_221230', 'BTCUSDT_230331', 'BTSUSDT', 'BTTUSDT', 'BZRXUSDT', 'C98USDT', 'CELOUSDT', 'CELRUSDT', 'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'CVCUSDT', 'CVXBUSD', 'CVXUSDT', 'DARUSDT', 'DASHUSDT', 'DEFIUS

In [2]:
# Get the current date
now = datetime.datetime.now()

# Format the date as a string in the desired format
date_string = now.strftime("%Y-%m")

# Print the date string
print(date_string)

idx = MONTH_ARRAY.index(date_string)
MONTH_ARRAY = MONTH_ARRAY[:idx + 1]

2023-01


## Set up the arrays of symbols, chart times, and months

In [3]:
print(SYMBOL_ARRAY.index('BTCUSDT'))

49


In [4]:
CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('5m')],
CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('1m')],
CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('15m')]

'15m'

In [5]:
# Set up the arrays of symbols, chart times, and months
symbols = [
    SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')],
    #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHUSDT')],
    #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHBUSD')],
    #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCBUSD')]
]
# symbols = [SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')]]

# chart_times = CHART_TIME_ARRAY
chart_times = [
    CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('5m')],
    CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('1m')],
    CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('15m')]
]
months = MONTH_ARRAY

## Download the data

In [ ]:
try:
    # Set up the ChromeOptions instance
    options = Options()

    # Set the download directory in the prefs dictionary
    prefs = {"download.default_directory": download_dir}
    options.add_experimental_option("prefs", prefs)

    # Set up the webdriver
    driver = webdriver.Chrome(options=options)

    # Open the webpage
    driver.get(BINANCE_URL)

    # Iterate over the symbols, chart times, and months
    for symbol in symbols:
        for chart_time in chart_times:
            for month in months:
                # Construct the link
                link = f"{BINANCE_URL}{symbol}/{chart_time}/{symbol}-{chart_time}-{month}.zip"
                #             print(link)
                # Open the link in a new tab
                driver.execute_script(f"window.open('{link}', '_blank');")

#     # Wait for the downloads to finish
#     driver.implicitly_wait(60)

# Close the webdriver
    driver.close()
except:
    pass

## Move to separate folders. Caution: run this only after the downloads are finished.

In [ ]:
for symbol in symbols:
    for chart_time in chart_times:
        # Create the new folder path
        new_folder_path = Path(download_dir) / f"{symbol}-{chart_time}"
        new_folder_path.mkdir(parents=True, exist_ok=True)
        # match pattern of the file
        pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")
        # list all files in the download directory
        files = os.listdir(download_dir)
        for file in files:
            if pattern.match(file):
                src = Path(download_dir) / file
                dst = new_folder_path / file
                src.rename(dst)

## Unzip the Zip files and concatenate the CSVs

### Move the monthly zips and csvs to a new folder.
#### Earlier, I wanted to delete them completely. But then I realized binance will not give more than 24 months of data at a time so now that we're in 2023, we wont get 2020 data. So I have decided not to delete those. (okay im not sure about this but i still dont want to risk)

In [6]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Set up an empty list for the data frames
        df_list = []

        # Compile the regular expression pattern
        pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

        # Create the new folder path for ZIP files
        new_zip_folder_path = os.path.join(download_dir,
                                           f"{symbol}-{chart_time}")

        # Create the new folder path for CSV files
        new_csv_folder_path = os.path.join(output_dir,
                                           f"{symbol}-{chart_time}")

        # Iterate over the files in the new zip folder
        for file in os.listdir(new_zip_folder_path):
            # Check if the file matches the pattern
            if pattern.match(file):
                # Construct the file path
                file_path = os.path.join(new_zip_folder_path, file)

                # Extract the ZIP file
                with zipfile.ZipFile(file_path, "r") as zip_ref:
                    zip_ref.extractall(new_csv_folder_path)

                # Construct the CSV file path
                csv_file_path = os.path.join(
                    new_csv_folder_path,
                    f"{symbol}-{chart_time}{file[-12:-4]}.csv")

                # Read the CSV file into a data frame, ignoring the headers
                df = pd.read_csv(csv_file_path, header=None)

                # Remove the first row (which contains the header)
                df = df.iloc[1:]

                # Add it to the list
                df_list.append(df)

        # Concatenate the data frames in the list
        df_final = pd.concat(df_list, ignore_index=True)

        # Read the headers from the first CSV file
        headers = pd.read_csv(csv_file_path, nrows=1).columns

        # Set the headers as the column names of the final dataframe
        df_final.columns = headers

        # Convert 'open_time' and 'close_time' columns to datetime
        df_final['open_time'] = pd.to_datetime(
            df_final['open_time'],
            unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
        df_final['close_time'] = pd.to_datetime(
            df_final['close_time'],
            unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')

        # Delete the 'ignore' column
        df_final = df_final.drop(['ignore'], axis=1)

        # Add a new column called 'entry' that will take previous close
        df_final['entry'] = df_final['close'].shift(1)
        # Set the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = os.path.join(new_csv_folder_path, file_name)

        # Write the data frame to the Excel file
        df_final.to_csv(file_path, index=False)

## Each candle is an entry. Below is the code to know if shorts wouldve won or longs wouldve won

In [ ]:
win_perc = 0.5
loss_perc = 0.4

for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        df["if_short"] = 0
        df["if_long"] = 0
        df["long_target"] = np.nan
        df["short_target"] = np.nan
        df["long_stop_loss"] = np.nan
        df["short_stop_loss"] = np.nan
        df["shorts_win_after"] = np.nan
        df["longs_win_after"] = np.nan

        for i in range(len(df)):
            if df["entry"][i]:
                long_target = df["entry"][i] * (1 + win_perc / 100)
                short_target = df["entry"][i] * (1 - win_perc / 100)
                long_stop_loss = df["entry"][i] * (1 - loss_perc / 100)
                short_stop_loss = df["entry"][i] * (1 + loss_perc / 100)
                df.loc[i, 'long_target'] = long_target
                df.loc[i, 'long_stop_loss'] = long_stop_loss
                for j in range(i, len(df)):
                    if df["high"][j] >= long_target:
                        df.loc[i, 'if_long'] = 1
                        df.loc[i, 'longs_win_after'] = j - i
                        break
                    if df["low"][j] <= long_stop_loss:
                        df.loc[i, 'if_long'] = -1
                        break
                df.loc[i, 'short_target'] = short_target
                df.loc[i, 'short_stop_loss'] = short_stop_loss
                for j in range(i, len(df)):
                    if df["low"][j] <= short_target:
                        df.loc[i, 'if_short'] = 1
                        df.loc[i, 'shorts_win_after'] = j - i
                        break
                    if df["high"][j] >= short_stop_loss:
                        df.loc[i, 'if_short'] = -1
                        break
        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

## Master cells to add different indicators to the dataframe and save it. Run it with caution because it takes really long


### 1. Overlap Studies

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(df['close'], timeperiod=5)
        df['DEMA'] = talib.DEMA(df['close'], timeperiod=30)
        df['EMA-50'] = talib.EMA(df['close'], timeperiod=50)
        df['EMA-200'] = talib.EMA(df['close'], timeperiod=200)
        df['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['close'])
        df['KAMA'] = talib.KAMA(df['close'], timeperiod=30)
        df['MA'] = talib.MA(df['close'], timeperiod=14)
#         df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
#         df['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
        df['MIDPOINT'] = talib.MIDPOINT(df['close'], timeperiod=14)
        df['MIDPRICE'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=14)
        df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)
        df['SAREXT'] = talib.SAREXT(df['high'], df['low'])
        df['SMA'] = talib.SMA(df['close'], timeperiod=14)
        df['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0)
        df['TEMA'] = talib.TEMA(df['close'])
        df['TRIMA'] = talib.TRIMA(df['close'])
        df['WMA'] = talib.WMA(df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 2. Momentum Indicators

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['ADX'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
        df['ADXR'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=14)
        df['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26)
        df['AROON_up'], df['AROON_down'] = talib.AROON(df['high'], df['low'], timeperiod=14)
        df['AROONOSC'] = talib.AROONOSC(df['high'], df['low'], timeperiod=14)
        df['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
        df['CCI'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
        df['CMO'] = talib.CMO(df['close'], timeperiod=14)
        df['DX'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=14)
        df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
#         df['MACDEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
#         df['MACDFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
        df['MFI'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14)
        df['MINUS_DI'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
        df['MINUS_DM'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=14)
        df['MOM'] = talib.MOM(df['close'], timeperiod=14)
        df['PLUS_DI'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
        df['PLUS_DM'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=14)
        df['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
        df['ROC'] = talib.ROC(df['close'], timeperiod=14)
        df['ROCP'] = talib.ROCP(df['close'], timeperiod=14)
        df['ROCR'] = talib.ROCR(df['close'], timeperiod=14)
        df['ROCR100'] = talib.ROCR100(df['close'], timeperiod=14)
        df['RSI'] = talib.RSI(df['close'], timeperiod=8)
#         df['STOCH'] = talib.STOCH(df['high'], df['low'], df['close'])
#         df['STOCHF'] = talib.STOCHF(df['high'], df['low'], df['close'])
#         df['STOCHRSI'] = talib.STOCHRSI(df['close'])
        df['TRIX'] = talib.TRIX(df['close'])
        df['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'])
        df['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 3. Volume Indicators

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
        df['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
        df['OBV'] = talib.OBV(df['close'], df['volume'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 4. Cycle Indicators

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
        df['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
        df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['close'])
#         df['HT_SINE'] = talib.HT_SINE(df['close'])
        df['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 5. Price Transform

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
        df['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
        df['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
        df['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 6. Volatility Indicators

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
        df['NATR'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=14)
        df['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 7. Pattern Recognition

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['CDL2CROWS'] = talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])
        df['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
        df['CDL3INSIDE'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
        df['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close'])
        df['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
        df['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
        df['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
        df['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'], penetration=0)
        df['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])
        df['CDLBELTHOLD'] = talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
        df['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close'])
        df['CDLCLOSINGMARUBOZU'] = talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])
        df['CDLCONCEALBABYSWALL'] = talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close'])
        df['CDLCOUNTERATTACK'] = talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close'])
        df['CDLDARKCLOUDCOVER'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'], penetration=0)
        df['CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
        df['CDLDOJISTAR'] = talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close'])
        df['CDLDRAGONFLYDOJI'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
        df['CDLENGULFING'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
        df['CDLEVENINGDOJISTAR'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
        df['CDLEVENINGSTAR'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
        df['CDLGAPSIDESIDEWHITE'] = talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close'])
        df['CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
        df['CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
        df['CDLHANGINGMAN'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
        df['CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close'])
        df['CDLHARAMICROSS'] = talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
        df['CDLHIGHWAVE'] = talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close'])
        df['CDLHIKKAKE'] = talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close'])
        df['CDLHIKKAKEMOD'] = talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close'])
        df['CDLHOMINGPIGEON'] = talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close'])
        df['CDLIDENTICAL3CROWS'] = talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close'])
        df['CDLINNECK'] = talib.CDLINNECK(df['open'], df['high'], df['low'], df['close'])
        df['CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
        df['CDLKICKING'] = talib.CDLKICKING(df['open'], df['high'], df['low'], df['close'])
        df['CDLKICKINGBYLENGTH'] = talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close'])
        df['CDLLADDERBOTTOM'] = talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close'])
        df['CDLLONGLEGGEDDOJI'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
        df['CDLLONGLINE'] = talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close'])
        df['CDLMARUBOZU'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
        df['CDLMATCHINGLOW'] = talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close'])
        df['CDLMATHOLD'] = talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0)
        df['CDLMORNINGDOJISTAR'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
        df['CDLMORNINGSTAR'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
        df['CDLONNECK'] = talib.CDLONNECK(df['open'], df['high'], df['low'], df['close'])
        df['CDLPIERCING'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
        df['CDLRICKSHAWMAN'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
        df['CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close'])
        df['CDLSEPARATINGLINES'] = talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close'])
        df['CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
        df['CDLSHORTLINE'] = talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close'])
        df['CDLSPINNINGTOP'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
        df['CDLSTALLEDPATTERN'] = talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close'])
        df['CDLSTICKSANDWICH'] = talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close'])
        df['CDLTAKURI'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
        df['CDLTASUKIGAP'] = talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close'])
        df['CDLTHRUSTING'] = talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close'])
        df['CDLTRISTAR'] = talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close'])
        df['CDLUNIQUE3RIVER'] = talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close'])
        df['CDLUPSIDEGAP2CROWS'] = talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close'])
        df['CDLXSIDEGAP3METHODS'] = talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

### 8. Statistic Functions

In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        print(df.dtypes)
        df['BETA'] = talib.BETA(df['high'], df['low'], timeperiod=5)
        df['CORREL'] = talib.CORREL(df['high'], df['low'], timeperiod=30)
        df['LINEARREG'] = talib.LINEARREG(df['close'])
        df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'])
        df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'])
        df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'])
#         df['STDDEV'] = df['close'].rolling(timeperiod).std()
        df['TSF'] = talib.TSF(df['close'])
        df['VAR'] = talib.VAR(df['close'])

        # Save the updated dataframe to the CSV file
        df.to_csv(file_path, index=False)

## Functions not part of any of the above 8 groups

In [ ]:
# # Iterate over the symbols and chart times
# for symbol in symbols:
#     for chart_time in chart_times:
#         # Construct the file name
#         file_name = f"{symbol}-{chart_time}.csv"

#         # Construct the file path
#         file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

#         # Read the CSV file into a dataframe
#         df = pd.read_csv(file_path)
#         print(df.dtypes)
#         df['ACOS'] = talib.ACOS(df['close'])
#         df['ADD'] = talib.ADD(df['close'], df['close'])
#         df['ASIN'] = talib.ASIN(df['close'])
#         df['ATAN'] = talib.ATAN(df['close'])
#         df['CEIL'] = talib.CEIL(df['close'])
#         df['COS'] = talib.COS(df['close'])
#         df['COSH'] = talib.COSH(df['close'])
#         df['DIV'] = talib.DIV(df['close'], df['close'].shift(10))
#         df['EXP'] = talib.EXP(df['close'])
#         df['FLOOR'] = talib.FLOOR(df['close'])
#         df['LN'] = talib.LN(df['close'])
#         df['LOG10'] = talib.LOG10(df['close'])
#         df['MAX'] = talib.MAX(df['close'], timeperiod=30)
#         df['MAXINDEX'] = talib.MAXINDEX(df['close'], timeperiod=30)
#         df['MIN'] = talib.MIN(df['close'], timeperiod=14)
#         df['MININDEX'] = talib.MININDEX(df['close'], timeperiod=14)
# #         df['MINMAX'] = talib.MINMAX(df['close'], timeperiod=14)
# #         df['MINMAXINDEX'] = talib.MINMAXINDEX(df['close'], timeperiod=14)
# #         df['MULT'] = talib.MULT(df['close'], df['close'], df['close'])
#         df['SIN'] = np.sin(df['close'])
#         df['SINH'] = np.sinh(df['close'])
#         df['SQRT'] = np.sqrt(df['close'])
#         df['SUB'] = df['close'].diff(1)
# #         df['SUM'] = df['close'].rolling(timeperiod).sum()
#         df['TAN'] = np.tan(df['close'])
#         df['TANH'] = np.tanh(df['close'])

#         # Save the updated dataframe to the CSV file
#         df.to_csv(file_path, index=False)

# Master Backup of all functions from TA-Lib Library

In [ ]:
# # Iterate over the symbols and chart times
# for symbol in symbols:
#     for chart_time in chart_times:
#         # Construct the file name
#         file_name = f"{symbol}-{chart_time}.csv"

#         # Construct the file path
#         file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

#         # Read the CSV file into a dataframe
#         df = pd.read_csv(file_path)
#         print(df.dtypes)
#         df['ACOS'] = talib.ACOS(df['close'])
#         df['AD'] = talib.AD(df['high'], df['low'], df['close'], df['volume'])
#         df['ADD'] = talib.ADD(df['close'], df['close'])
#         df['ADOSC'] = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
#         df['ADX'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
#         df['ADXR'] = talib.ADXR(df['high'], df['low'], df['close'], timeperiod=14)
#         df['APO'] = talib.APO(df['close'], fastperiod=12, slowperiod=26)
#         df['AROON_up'], df['AROON_down'] = talib.AROON(df['high'], df['low'], timeperiod=14)
#         df['AROONOSC'] = talib.AROONOSC(df['high'], df['low'], timeperiod=14)
#         df['ASIN'] = talib.ASIN(df['close'])
#         df['ATAN'] = talib.ATAN(df['close'])
#         df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
#         df['AVGPRICE'] = talib.AVGPRICE(df['open'], df['high'], df['low'], df['close'])
#         df['BB_upper'], df['BB_middle'], df['BB_lower'] = talib.BBANDS(df['close'], timeperiod=5)
#         df['BETA'] = talib.BETA(df['high'], df['low'], timeperiod=5)
#         df['BOP'] = talib.BOP(df['open'], df['high'], df['low'], df['close'])
#         df['CCI'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
#         df['CDL2CROWS'] = talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])
#         df['CDL3BLACKCROWS'] = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
#         df['CDL3INSIDE'] = talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close'])
#         df['CDL3LINESTRIKE'] = talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close'])
#         df['CDL3OUTSIDE'] = talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close'])
#         df['CDL3STARSINSOUTH'] = talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close'])
#         df['CDL3WHITESOLDIERS'] = talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close'])
#         df['CDLABANDONEDBABY'] = talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'], penetration=0)
#         df['CDLADVANCEBLOCK'] = talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close'])
#         df['CDLBELTHOLD'] = talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close'])
#         df['CDLBREAKAWAY'] = talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close'])
#         df['CDLCLOSINGMARUBOZU'] = talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close'])
#         df['CDLCONCEALBABYSWALL'] = talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close'])
#         df['CDLCOUNTERATTACK'] = talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close'])
#         df['CDLDARKCLOUDCOVER'] = talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'], penetration=0)
#         df['CDLDOJI'] = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
#         df['CDLDOJISTAR'] = talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close'])
#         df['CDLDRAGONFLYDOJI'] = talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
#         df['CDLENGULFING'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
#         df['CDLEVENINGDOJISTAR'] = talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
#         df['CDLEVENINGSTAR'] = talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0)
#         df['CDLGAPSIDESIDEWHITE'] = talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close'])
#         df['CDLGRAVESTONEDOJI'] = talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHAMMER'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHANGINGMAN'] = talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHARAMI'] = talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHARAMICROSS'] = talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHIGHWAVE'] = talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHIKKAKE'] = talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHIKKAKEMOD'] = talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close'])
#         df['CDLHOMINGPIGEON'] = talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close'])
#         df['CDLIDENTICAL3CROWS'] = talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close'])
#         df['CDLINNECK'] = talib.CDLINNECK(df['open'], df['high'], df['low'], df['close'])
#         df['CDLINVERTEDHAMMER'] = talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close'])
#         df['CDLKICKING'] = talib.CDLKICKING(df['open'], df['high'], df['low'], df['close'])
#         df['CDLKICKINGBYLENGTH'] = talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close'])
#         df['CDLLADDERBOTTOM'] = talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close'])
#         df['CDLLONGLEGGEDDOJI'] = talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close'])
#         df['CDLLONGLINE'] = talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close'])
#         df['CDLMARUBOZU'] = talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close'])
#         df['CDLMATCHINGLOW'] = talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close'])
#         df['CDLMATHOLD'] = talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0)
#         df['CDLMORNINGDOJISTAR'] = talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close'])
#         df['CDLMORNINGSTAR'] = talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close'])
#         df['CDLONNECK'] = talib.CDLONNECK(df['open'], df['high'], df['low'], df['close'])
#         df['CDLPIERCING'] = talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close'])
#         df['CDLRICKSHAWMAN'] = talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close'])
#         df['CDLRISEFALL3METHODS'] = talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close'])
#         df['CDLSEPARATINGLINES'] = talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close'])
#         df['CDLSHOOTINGSTAR'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
#         df['CDLSHORTLINE'] = talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close'])
#         df['CDLSPINNINGTOP'] = talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close'])
#         df['CDLSTALLEDPATTERN'] = talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close'])
#         df['CDLSTICKSANDWICH'] = talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close'])
#         df['CDLTAKURI'] = talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close'])
#         df['CDLTASUKIGAP'] = talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close'])
#         df['CDLTHRUSTING'] = talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close'])
#         df['CDLTRISTAR'] = talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close'])
#         df['CDLUNIQUE3RIVER'] = talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close'])
#         df['CDLUPSIDEGAP2CROWS'] = talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close'])
#         df['CDLXSIDEGAP3METHODS'] = talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close'])
#         df['CEIL'] = talib.CEIL(df['close'])
#         df['CMO'] = talib.CMO(df['close'], timeperiod=14)
#         df['CORREL'] = talib.CORREL(df['high'], df['low'], timeperiod=30)
#         df['COS'] = talib.COS(df['close'])
#         df['COSH'] = talib.COSH(df['close'])
#         df['DEMA'] = talib.DEMA(df['close'], timeperiod=30)
#         df['DIV'] = talib.DIV(df['close'], df['close'].shift(10))
#         df['DX'] = talib.DX(df['high'], df['low'], df['close'], timeperiod=14)
#         df['EMA-50'] = talib.EMA(df['close'], timeperiod=50)
#         df['EMA-200'] = talib.EMA(df['close'], timeperiod=200)
#         df['EXP'] = talib.EXP(df['close'])
#         df['FLOOR'] = talib.FLOOR(df['close'])
#         df['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['close'])
#         df['HT_DCPHASE'] = talib.HT_DCPHASE(df['close'])
#         df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['close'])
# #         df['HT_SINE'] = talib.HT_SINE(df['close'])
#         df['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['close'])
#         df['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['close'])
#         df['KAMA'] = talib.KAMA(df['close'], timeperiod=30)
#         df['LINEARREG'] = talib.LINEARREG(df['close'])
#         df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['close'])
#         df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['close'])
#         df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['close'])
#         df['LN'] = talib.LN(df['close'])
#         df['LOG10'] = talib.LOG10(df['close'])
#         df['MA'] = talib.MA(df['close'], timeperiod=14)
#         df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
# #         df['MACDEXT'] = talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
# #         df['MACDFIX'] = talib.MACDFIX(df['close'], signalperiod=9)
# #         df['MAMA'], df['FAMA'] = talib.MAMA(df['close'], fastlimit=0, slowlimit=0)
# #         df['MAVP'] = talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0)
#         df['MAX'] = talib.MAX(df['close'], timeperiod=30)
#         df['MAXINDEX'] = talib.MAXINDEX(df['close'], timeperiod=30)
#         df['MEDPRICE'] = talib.MEDPRICE(df['high'], df['low'])
#         df['MFI'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14)
#         df['MIDPOINT'] = talib.MIDPOINT(df['close'], timeperiod=14)
#         df['MIDPRICE'] = talib.MIDPRICE(df['high'], df['low'], timeperiod=14)
#         df['MIN'] = talib.MIN(df['close'], timeperiod=14)
#         df['MININDEX'] = talib.MININDEX(df['close'], timeperiod=14)
# #         df['MINMAX'] = talib.MINMAX(df['close'], timeperiod=14)
# #         df['MINMAXINDEX'] = talib.MINMAXINDEX(df['close'], timeperiod=14)
#         df['MINUS_DI'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
#         df['MINUS_DM'] = talib.MINUS_DM(df['high'], df['low'], timeperiod=14)
#         df['MOM'] = talib.MOM(df['close'], timeperiod=14)
# #         df['MULT'] = talib.MULT(df['close'], df['close'], df['close'])
#         df['NATR'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=14)
#         df['OBV'] = talib.OBV(df['close'], df['volume'])
#         df['PLUS_DI'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
#         df['PLUS_DM'] = talib.PLUS_DM(df['high'], df['low'], timeperiod=14)
#         df['PPO'] = talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
#         df['ROC'] = talib.ROC(df['close'], timeperiod=14)
#         df['ROCP'] = talib.ROCP(df['close'], timeperiod=14)
#         df['ROCR'] = talib.ROCR(df['close'], timeperiod=14)
#         df['ROCR100'] = talib.ROCR100(df['close'], timeperiod=14)
#         df['RSI'] = talib.RSI(df['close'], timeperiod=8)
#         df['SAR'] = talib.SAR(df['high'], df['low'], acceleration=0, maximum=0)
#         df['SAREXT'] = talib.SAREXT(df['high'], df['low'])
#         df['SIN'] = np.sin(df['close'])
#         df['SINH'] = np.sinh(df['close'])
#         df['SMA'] = talib.SMA(df['close'], timeperiod=14)
#         df['SQRT'] = np.sqrt(df['close'])
# #         df['STDDEV'] = df['close'].rolling(timeperiod).std()
# #         df['STOCH'] = talib.STOCH(df['high'], df['low'], df['close'])
# #         df['STOCHF'] = talib.STOCHF(df['high'], df['low'], df['close'])
# #         df['STOCHRSI'] = talib.STOCHRSI(df['close'])
#         df['SUB'] = df['close'].diff(1)
# #         df['SUM'] = df['close'].rolling(timeperiod).sum()
#         df['T3'] = talib.T3(df['close'], timeperiod=5, vfactor=0)
#         df['TAN'] = np.tan(df['close'])
#         df['TANH'] = np.tanh(df['close'])
#         df['TEMA'] = talib.TEMA(df['close'])
#         df['TRANGE'] = talib.TRANGE(df['high'], df['low'], df['close'])
#         df['TRIMA'] = talib.TRIMA(df['close'])
#         df['TRIX'] = talib.TRIX(df['close'])
#         df['TSF'] = talib.TSF(df['close'])
#         df['TYPPRICE'] = talib.TYPPRICE(df['high'], df['low'], df['close'])
#         df['ULTOSC'] = talib.ULTOSC(df['high'], df['low'], df['close'])
#         df['VAR'] = talib.VAR(df['close'])
#         df['WCLPRICE'] = talib.WCLPRICE(df['high'], df['low'], df['close'])
#         df['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'])
#         df['WMA'] = talib.WMA(df['close'])

#         # Save the updated dataframe to the CSV file
#         df.to_csv(file_path, index=False)

## The below cell is a backup written because the above code gave performance issues so it suggested me to use pd.concat. but then i dont know how to make it work.

In [ ]:
# # Iterate over the symbols and chart times
# for symbol in symbols:
#     for chart_time in chart_times:
#         # Construct the file name
#         file_name = f"{symbol}-{chart_time}.csv"

#         # Construct the file path
#         file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

#         # Read the CSV file into a dataframe
#         df = pd.read_csv(file_path)
#         print(df.dtypes)
#         df = pd.concat([df,
#         pd.DataFrame(talib.ACOS(df['close']),df['ACOS']),
#         pd.DataFrame(talib.AD(df['high'], df['low'], df['close'], df['volume']), df['AD']),
#         pd.DataFrame(talib.ADD(df['close'], df['close']), df['ADD']),
#         pd.DataFrame(talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10), df['ADOSC']),
#         pd.DataFrame(talib.ADX(df['high'], df['low'], df['close'], timeperiod=14), df['ADX']),
#         pd.DataFrame(talib.ADXR(df['high'], df['low'], df['close'], timeperiod=14), df['ADXR']),
#         pd.DataFrame(talib.APO(df['close'], fastperiod=12, slowperiod=26),df['APO']),
#         pd.DataFrame(talib.AROON(df['high'], df['low'], timeperiod=14), df['AROON_up']),
#         pd.DataFrame(talib.AROON(df['high'], df['low'], timeperiod=14), df['AROON_down']),
#         pd.DataFrame(talib.AROONOSC(df['high'], df['low'], timeperiod=14),df['AROONOSC']),
#         pd.DataFrame(talib.ASIN(df['close']),df['ASIN']),
#         pd.DataFrame(talib.ATAN(df['close']),df['ATAN']),
#         pd.DataFrame(talib.ATR(df['high'], df['low'], df['close'], timeperiod=14),df['ATR']),
#         pd.DataFrame(talib.AVGPRICE(df['open'], df['high'], df['low'], df['close']),df['AVGPRICE']),
#         pd.DataFrame(talib.BBANDS(df['close'], timeperiod=5), df['BB_upper']),
#         pd.DataFrame(talib.BBANDS(df['close'], timeperiod=5), df['BB_middle']),
#         pd.DataFrame(talib.BBANDS(df['close'], timeperiod=5), df['BB_lower']),
#         pd.DataFrame(talib.BETA(df['high'], df['low'], timeperiod=5), df['BETA']),
#         pd.DataFrame(talib.BOP(df['open'], df['high'], df['low'], df['close']),df['BOP']),
#         pd.DataFrame(talib.CCI(df['high'], df['low'], df['close'], timeperiod=14), df['CCI']),
#         pd.DataFrame(talib.CDL2CROWS(df['open'], df['high'], df['low'], df['close']),df['CDL2CROWS']),
#         pd.DataFrame(talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close']),df['CDL3BLACKCROWS']),
#         pd.DataFrame(talib.CDL3INSIDE(df['open'], df['high'], df['low'], df['close']),df['CDL3INSIDE']),
#         pd.DataFrame(talib.CDL3LINESTRIKE(df['open'], df['high'], df['low'], df['close']),df['CDL3LINESTRIKE']),
#         pd.DataFrame(talib.CDL3OUTSIDE(df['open'], df['high'], df['low'], df['close']),df['CDL3OUTSIDE']).
#         pd.DataFrame(talib.CDL3STARSINSOUTH(df['open'], df['high'], df['low'], df['close']),df['CDL3STARSINSOUTH']),
#         pd.DataFrame(talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close']),df['CDL3WHITESOLDIERS']),
#         pd.DataFrame(talib.CDLABANDONEDBABY(df['open'], df['high'], df['low'], df['close'], penetration=0),df['CDLABANDONEDBABY']),
#         pd.DataFrame(talib.CDLADVANCEBLOCK(df['open'], df['high'], df['low'], df['close']),df['CDLADVANCEBLOCK']),
#         pd.DataFrame(talib.CDLBELTHOLD(df['open'], df['high'], df['low'], df['close']),df['CDLBELTHOLD']),
#         pd.DataFrame(talib.CDLBREAKAWAY(df['open'], df['high'], df['low'], df['close']),df['CDLBREAKAWAY']),
#         pd.DataFrame(talib.CDLCLOSINGMARUBOZU(df['open'], df['high'], df['low'], df['close']),df['CDLCLOSINGMARUBOZU']),
#         pd.DataFrame(talib.CDLCONCEALBABYSWALL(df['open'], df['high'], df['low'], df['close']),df['CDLCONCEALBABYSWALL']),
#         pd.DataFrame(talib.CDLCOUNTERATTACK(df['open'], df['high'], df['low'], df['close']),df['CDLCOUNTERATTACK']),
#         pd.DataFrame(talib.CDLDARKCLOUDCOVER(df['open'], df['high'], df['low'], df['close'], penetration=0),df['CDLDARKCLOUDCOVER']),
#         pd.DataFrame(talib.CDLDOJI(df['open'], df['high'], df['low'], df['close']),df['CDLDOJI']),
#         pd.DataFrame(talib.CDLDOJISTAR(df['open'], df['high'], df['low'], df['close']),df['CDLDOJISTAR']),
#         pd.DataFrame(talib.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close']),df['CDLDRAGONFLYDOJI']),
#         pd.DataFrame(talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']),df['CDLENGULFING']),
#         pd.DataFrame(talib.CDLEVENINGDOJISTAR(df['open'], df['high'], df['low'], df['close'], penetration=0),df['CDLEVENINGDOJISTAR']),
#         pd.DataFrame(talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close'], penetration=0),df['CDLEVENINGSTAR']),
#         pd.DataFrame(talib.CDLGAPSIDESIDEWHITE(df['open'], df['high'], df['low'], df['close']),df['CDLGAPSIDESIDEWHITE']),
#         pd.DataFrame(talib.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close']),df['CDLGRAVESTONEDOJI']),
#         pd.DataFrame(talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close']),df['CDLHAMMER']),
#         pd.DataFrame(talib.CDLHANGINGMAN(df['open'], df['high'], df['low'], df['close']),df['CDLHANGINGMAN']),
#         pd.DataFrame(talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close']),df['CDLHARAMI']),
#         pd.DataFrame(talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close']),df['CDLHARAMICROSS']),
#         pd.DataFrame(talib.CDLHIGHWAVE(df['open'], df['high'], df['low'], df['close']),df['CDLHIGHWAVE']),
#         pd.DataFrame(talib.CDLHIKKAKE(df['open'], df['high'], df['low'], df['close']),df['CDLHIKKAKE']),
#         pd.DataFrame(talib.CDLHIKKAKEMOD(df['open'], df['high'], df['low'], df['close']),df['CDLHIKKAKEMOD']),
#         pd.DataFrame(talib.CDLHOMINGPIGEON(df['open'], df['high'], df['low'], df['close']),df['CDLHOMINGPIGEON']),
#         pd.DataFrame(talib.CDLIDENTICAL3CROWS(df['open'], df['high'], df['low'], df['close']),df['CDLIDENTICAL3CROWS']),
#         pd.DataFrame(talib.CDLINNECK(df['open'], df['high'], df['low'], df['close']),df['CDLINNECK']),
#         pd.DataFrame(talib.CDLINVERTEDHAMMER(df['open'], df['high'], df['low'], df['close']),df['CDLINVERTEDHAMMER']),
#         pd.DataFrame(talib.CDLKICKING(df['open'], df['high'], df['low'], df['close']), df['CDLKICKING']),
#         pd.DataFrame(talib.CDLKICKINGBYLENGTH(df['open'], df['high'], df['low'], df['close']),df['CDLKICKINGBYLENGTH']),
#         pd.DataFrame(talib.CDLLADDERBOTTOM(df['open'], df['high'], df['low'], df['close']),df['CDLLADDERBOTTOM']),
#         pd.DataFrame(talib.CDLLONGLEGGEDDOJI(df['open'], df['high'], df['low'], df['close']),df['CDLLONGLEGGEDDOJI']),
#         pd.DataFrame(talib.CDLLONGLINE(df['open'], df['high'], df['low'], df['close']),df['CDLLONGLINE']),
#         pd.DataFrame(talib.CDLMARUBOZU(df['open'], df['high'], df['low'], df['close']),df['CDLMARUBOZU']),
#         pd.DataFrame(talib.CDLMATCHINGLOW(df['open'], df['high'], df['low'], df['close']),df['CDLMATCHINGLOW']),
#         pd.DataFrame(talib.CDLMATHOLD(df['open'], df['high'], df['low'], df['close'], penetration=0),df['CDLMATHOLD']),
#         pd.DataFrame(talib.CDLMORNINGDOJISTAR(df['open'], df['high'], df['low'], df['close']),df['CDLMORNINGDOJISTAR']),
#         pd.DataFrame(talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close']),df['CDLMORNINGSTAR']),
#         pd.DataFrame(talib.CDLONNECK(df['open'], df['high'], df['low'], df['close']),df['CDLONNECK']),
#         pd.DataFrame(talib.CDLPIERCING(df['open'], df['high'], df['low'], df['close']),df['CDLPIERCING']),
#         pd.DataFrame(talib.CDLRICKSHAWMAN(df['open'], df['high'], df['low'], df['close']),df['CDLRICKSHAWMAN']),
#         pd.DataFrame(talib.CDLRISEFALL3METHODS(df['open'], df['high'], df['low'], df['close']),df['CDLRISEFALL3METHODS']),
#         pd.DataFrame(talib.CDLSEPARATINGLINES(df['open'], df['high'], df['low'], df['close']),columns=['CDLSEPARATINGLINES']),
#         pd.DataFrame(talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close']),columns=['CDLSHOOTINGSTAR']),
#         pd.DataFrame(talib.CDLSHORTLINE(df['open'], df['high'], df['low'], df['close']),columns=['CDLSHORTLINE']),
#         pd.DataFrame(talib.CDLSPINNINGTOP(df['open'], df['high'], df['low'], df['close']),columns=['CDLSPINNINGTOP']),
#         pd.DataFrame(talib.CDLSTALLEDPATTERN(df['open'], df['high'], df['low'], df['close']),columns=['CDLSTALLEDPATTERN']),
#         pd.DataFrame(talib.CDLSTICKSANDWICH(df['open'], df['high'], df['low'], df['close']),columns=['CDLSTICKSANDWICH']),
#         pd.DataFrame(talib.CDLTAKURI(df['open'], df['high'], df['low'], df['close']),columns=['CDLTAKURI']),
#         pd.DataFrame(talib.CDLTASUKIGAP(df['open'], df['high'], df['low'], df['close']),columns=['CDLTASUKIGAP']),
#         pd.DataFrame(talib.CDLTHRUSTING(df['open'], df['high'], df['low'], df['close']),df['CDLTHRUSTING']),
#         pd.DataFrame(talib.CDLTRISTAR(df['open'], df['high'], df['low'], df['close']),df['CDLTRISTAR']),
#         pd.DataFrame(talib.CDLUNIQUE3RIVER(df['open'], df['high'], df['low'], df['close']),df['CDLUNIQUE3RIVER']),
#         pd.DataFrame(talib.CDLUPSIDEGAP2CROWS(df['open'], df['high'], df['low'], df['close']),df['CDLUPSIDEGAP2CROWS']),
#         pd.DataFrame(talib.CDLXSIDEGAP3METHODS(df['open'], df['high'], df['low'], df['close']),df['CDLXSIDEGAP3METHODS']),
#         pd.DataFrame(talib.CEIL(df['close']),df['CEIL']),
#         pd.DataFrame(talib.CMO(df['close'], timeperiod=14),df['CMO']),
#         pd.DataFrame(talib.CORREL(df['high'], df['low'], timeperiod=30),df['CORREL']),
#         pd.DataFrame(talib.COS(df['close']),df['COS']),
#         pd.DataFrame(talib.COSH(df['close']),df['COSH']),
#         pd.DataFrame(talib.DEMA(df['close'], timeperiod=30),df['DEMA']),
#         pd.DataFrame(talib.DIV(df['close'], df['close'].shift(10)),df['DIV']),
#         pd.DataFrame(talib.DX(df['high'], df['low'], df['close'], timeperiod=14),df['DX']),
#         pd.DataFrame(talib.EMA(df['close'], timeperiod=30),df['EMA']),
#         pd.DataFrame(talib.EXP(df['close']),df['EXP']),
#         pd.DataFrame(talib.FLOOR(df['close']),df['FLOOR']),
#         pd.DataFrame(talib.HT_DCPERIOD(df['close']),df['HT_DCPERIOD']),
#         pd.DataFrame(talib.HT_DCPHASE(df['close']),df['HT_DCPHASE']),
#         pd.DataFrame(talib.HT_PHASOR(df['close']),df['HT_PHASOR_inphase']),
#         pd.DataFrame(talib.HT_PHASOR(df['close']),df['HT_PHASOR_quadrature']),
# #         pd.DataFrame(talib.HT_SINE(df['close']),df['HT_SINE']),
#         pd.DataFrame(talib.HT_TRENDLINE(df['close']),df['HT_TRENDLINE']),
#         pd.DataFrame(talib.HT_TRENDMODE(df['close']),df['HT_TRENDMODE']),
#         pd.DataFrame(talib.KAMA(df['close'], timeperiod=30),df['KAMA']),
#         pd.DataFrame(talib.LINEARREG(df['close']),df['LINEARREG']),
#         pd.DataFrame(talib.LINEARREG_ANGLE(df['close']),df['LINEARREG_ANGLE']),
#         pd.DataFrame(talib.LINEARREG_INTERCEPT(df['close']),df['LINEARREG_INTERCEPT']),
#         pd.DataFrame(talib.LINEARREG_SLOPE(df['close']),df['LINEARREG_SLOPE']),
#         pd.DataFrame(talib.LN(df['close']),df['LN']),
#         pd.DataFrame(talib.LOG10(df['close']),df['LOG10']),
#         pd.DataFrame(talib.MA(df['close'], timeperiod=14),df['MA']),
#         pd.DataFrame(talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9),df['MACD']),
#         pd.DataFrame(talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9),df['MACD_signal']),
#         pd.DataFrame(talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9),df['MACD_hist']),
# #         pd.DataFrame(talib.MACDEXT(df['close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0),df['MACDEXT']),
# #         pd.DataFrame(talib.MACDFIX(df['close'], signalperiod=9),df['MACDFIX']),
# #         pd.DataFrame(talib.MAMA(df['close'], fastlimit=0, slowlimit=0),df['MAMA']),
# #         pd.DataFrame(talib.MAMA(df['close'], fastlimit=0, slowlimit=0),df['FAMA']),
# #         pd.DataFrame(talib.MAVP(df['close'], periods=None, minperiod=2, maxperiod=30, matype=0),df['MAVP']),
#         pd.DataFrame(talib.MAX(df['close'], timeperiod=30),df['MAX']),
#         pd.DataFrame(talib.MAXINDEX(df['close'], timeperiod=30),df['MAXINDEX']),
#         pd.DataFrame(talib.MEDPRICE(df['high'], df['low']),df['MEDPRICE']),
#         pd.DataFrame(talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14),df['MFI']),
#         pd.DataFrame(talib.MIDPOINT(df['close'], timeperiod=14),df['MIDPOINT']),
#         pd.DataFrame(talib.MIDPRICE(df['high'], df['low'], timeperiod=14),df['MIDPRICE']),
#         pd.DataFrame(talib.MIN(df['close'], timeperiod=14),df['MIN']),
#         pd.DataFrame(talib.MININDEX(df['close'], timeperiod=14),df['MININDEX']),
# #         pd.DataFrame(talib.MINMAX(df['close'], timeperiod=14),df['MINMAX']),
# #         pd.DataFrame(talib.MINMAXINDEX(df['close'], timeperiod=14),df['MINMAXINDEX']),
#         pd.DataFrame(talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=14),df['MINUS_DI']),
#         pd.DataFrame(talib.MINUS_DM(df['high'], df['low'], timeperiod=14),df['MINUS_DM']),
#         pd.DataFrame(talib.MOM(df['close'], timeperiod=14),df['MOM']),
# #         pd.DataFrame(talib.MULT(df['close'], df['close'], df['close']),df['MULT']),
#         pd.DataFrame(talib.NATR(df['high'], df['low'], df['close'], timeperiod=14),df['NATR']),
#         pd.DataFrame(talib.OBV(df['close'], df['volume']),df['OBV']),
#         pd.DataFrame(talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=14),df['PLUS_DI']),
#         pd.DataFrame(talib.PLUS_DM(df['high'], df['low'], timeperiod=14),df['PLUS_DM']),
#         pd.DataFrame(talib.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0),df['PPO']),
#         pd.DataFrame(talib.ROC(df['close'], timeperiod=14),df['ROC']),
#         pd.DataFrame(talib.ROCP(df['close'], timeperiod=14),df['ROCP']),
#         pd.DataFrame(talib.ROCR(df['close'], timeperiod=14),df['ROCR']),
#         pd.DataFrame(talib.ROCR100(df['close'], timeperiod=14),df['ROCR100']),
#         pd.DataFrame(talib.RSI(df['close'], timeperiod=8),df['RSI']),
#         pd.DataFrame(talib.SAR(df['high'], df['low'], acceleration=0, maximum=0),df['SAR']),
#         pd.DataFrame(talib.SAREXT(df['high'], df['low']),df['SAREXT']),
#         pd.DataFrame(np.sin(df['close']),df['SIN']) ,
#         pd.DataFrame(np.sinh(df['close']),df['SINH']),
#         pd.DataFrame(talib.SMA(df['close'], timeperiod=14),df['SMA']),
#         pd.DataFrame(np.sqrt(df['close']), df['SQRT']),
# #         df['STDDEV'] = df['close'].rolling(timeperiod).std(),
# #         pd.DataFrame(talib.STOCH(df['high'], df['low'], df['close']),df['STOCH']),
# #         pd.DataFrame(talib.STOCHF(df['high'], df['low'], df['close']),df['STOCHF']),
# #         pd.DataFrame(talib.STOCHRSI(df['close']),df['STOCHRSI']),
#         pd.DataFrame(df['close'].diff(1), df['SUB']),
# #         pd.DataFrame(df['close'].rolling(timeperiod).sum(),df['SUM']),
#         pd.DataFrame(talib.T3(df['close'], timeperiod=5, vfactor=0),df['T3']),
#         pd.DataFrame(np.tan(df['close']),df['TAN']),
#         pd.DataFrame(np.tanh(df['close']),df['TANH']),
#         pd.DataFrame(talib.TEMA(df['close']),df['TEMA']),
#         pd.DataFrame(talib.TRANGE(df['high'], df['low'], df['close']),df['TRANGE']),
#         pd.DataFrame(talib.TRIMA(df['close']),df['TRIMA']),
#         pd.DataFrame(talib.TRIX(df['close']),df['TRIX']),
#         pd.DataFrame(talib.TSF(df['close']),df['TSF']),
#         pd.DataFrame(talib.TYPPRICE(df['high'], df['low'], df['close']),df['TYPPRICE']),
#         pd.DataFrame(talib.ULTOSC(df['high'], df['low'], df['close']),df['ULTOSC']),
#         pd.DataFrame(talib.VAR(df['close']),df['VAR']),
#         pd.DataFrame(talib.WCLPRICE(df['high'], df['low'], df['close']),df['WCLPRICE']),
#         pd.DataFrame(talib.WILLR(df['high'], df['low'], df['close']),df['WILLR']),
#         pd.DataFrame(talib.WMA(df['close']),df['WMA'])])
#         display(df)
#         # Save the updated dataframe to the CSV file
# #         df.to_csv(file_path, index=False)

In [ ]:
indicators_master_list = talib.get_functions()
indicators_list = list(set(indicators_master_list))
indicators_list.sort()
for indicator in indicators_list:
    print(indicator)


### Troubleshooting utilities. Can be put in another notebook 


In [ ]:
column_names = pd.read_csv(csv_file_path, nrows=0).columns
print(column_names)

In [ ]:
# Set the chart time and symbol
chart_time = "5m"
symbol = "BTCBUSD"

# Set the file name
file_name = f"{symbol}-{chart_time}.csv"

# Construct the file path
file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

# Read the CSV file into a dataframe
df = pd.read_csv(file_path)

# Iterate over the rows of the dataframe
for i, row in df.iterrows():
    try:
        # Convert the 'open' column to float
        open_value = float(row['open'])
    except ValueError:
        # Print the value that caused the error
        print(row['1'])